In [ ]:
!pip install azure-functions
!pip install azure-core
!pip install azure-cosmos
!pip install openai
!pip install numpy
!pip install requests
!pip install pandas
!pip install azure-storage-blob 
!pip install azure-identity
!pip install smart_open
!pip install tenacity
!pip install pinecone-client
!pip install redis
!pip install tiktoken
!pip install azure-storage-file-share
!pip install python-dotenv
!pip install azure-search-documents

In [1]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

import shutil
import sys
sys.path.append('./utils')

# from utils import redis_helpers
# from utils import helpers
# from utils import language
# from utils import openai_helpers
# # from utils import storage
# from utils import bot_helpers

%load_ext autoreload
%autoreload 2

In [2]:
#### Ingest all knowledge base documents
from utils import cogsearch_helpers

cogsearch_helpers.ingest_kb()

Index km-openai Deleted
Index km-openai created
Deleted Skillset - km-openai-skills
Created new Skillset - km-openai-skills
Deleted Indexer - km-openai-indexer
Deleted Data Source - km-openai-skills
Created new Data Source Connection - km-openai-docs
Created new Indexer - km-openai-indexer
Running Indexer km-openai-indexer


In [2]:
#### Reset Index in Redis
from utils import redis_helpers

reset_index = False

if reset_index:
    ADA_002_MODEL_MAX_TOKENS = os.environ["ADA_002_MODEL_MAX_TOKENS"] 

    redis_conn = redis_helpers.get_new_conn()
    redis_helpers.redis_reset_index(redis_conn)

In [7]:
RESET_REDIS = True

if RESET_REDIS:
    redis_helpers.redis_reset_index(ADA_002_EMBEDDING_MODEL)


In [ ]:
import json


CHOSEN_EMB_MODEL   = os.environ['CHOSEN_EMB_MODEL']
SMALL_EMB_TOKEN_NUM  = int(os.environ['SMALL_EMB_TOKEN_NUM'])
MEDIUM_EMB_TOKEN_NUM  = int(os.environ['MEDIUM_EMB_TOKEN_NUM'])
LARGE_EMB_TOKEN_NUM  = int(os.environ['LARGE_EMB_TOKEN_NUM'])

for item in os.listdir("dump"):
    path = os.path.join("dump", item)

    with open(path, 'r') as openfile:
        data = json.load(openfile)
        break

emb_documents = []

emb_documents += helpers.generate_embeddings(data, CHOSEN_EMB_MODEL, SMALL_EMB_TOKEN_NUM,  text_suffix = 'S')

if MEDIUM_EMB_TOKEN_NUM != 0:
    emb_documents += helpers.generate_embeddings(data, CHOSEN_EMB_MODEL, MEDIUM_EMB_TOKEN_NUM, text_suffix = 'M')

if LARGE_EMB_TOKEN_NUM != 0:
    emb_documents += helpers.generate_embeddings(data, CHOSEN_EMB_MODEL, LARGE_EMB_TOKEN_NUM,  text_suffix = 'L')

helpers.load_embedding_docs_in_redis(emb_documents)

In [ ]:
emb_documents = []

emb_documents += helpers.generate_embeddings_from_json_docs('dump', ADA_002_EMBEDDING_MODEL, ADA_002_MODEL_MAX_TOKENS, text_suffix='XL', limit=-1)

print(f"Generated {len(emb_documents)} embeddings.")
helpers.save_embdding_docs_to_pkl(emb_documents, "test.pkl")

In [24]:
emb_documents = helpers.load_embedding_docs_from_pkl("test.pkl")
helpers.load_embedding_docs_in_redis(emb_documents)

Loading 141 embeddings into Redis


In [ ]:
queries = [
        "in which classes did the Danish sailors qualify?",
        "what are the reviews of the Lost City hotel?"
    ]


for q in queries:
    output = bot_helpers.openai_interrogate_text(q, DAVINCI_003_COMPLETIONS_MODEL, ADA_002_EMBEDDING_MODEL, 5, False)
    print("\n\n", output, '\n\n\n###############################')
    break

